# **INSTALLING THE REQUIRED LIBRARIES**

In [3]:
!nvidia-smi

Sat Apr  6 01:20:53 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 2.2 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.26.1
    Uninstalling accelerate-0.26.1:
      Successfully uninstalled accelerate-0.26.1
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 35.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.0
    Uninstalling transformers-4.37.0:
      Successfully uninstalled transformers-4.37.0
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install neattext

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 401.6 kB/s eta 0:00:000:010:010m
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [10]:
import torch

if torch.cuda.is_available():
    # Print the current device
    device = torch.device("cuda")
    print("Current Device:", device)

    num_gpus = torch.cuda.device_count()

    for i in range(num_gpus):
        print("Device {}: {}".format(i, torch.cuda.get_device_name(i)))
else:
    print("CUDA is not available. Using CPU.")

Current Device: cuda
Device 0: Tesla T4
Device 1: Tesla T4


# **IMPORTING THE LIBRARIES**

In [11]:
import os
import re
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import neattext.functions as nfx
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, KFold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from imblearn.under_sampling import RandomUnderSampler

In [12]:
dir(nfx)

['BTC_ADDRESS_REGEX',
 'CURRENCY_REGEX',
 'CURRENCY_SYMB_REGEX',
 'Counter',
 'DATE_REGEX',
 'EMAIL_REGEX',
 'EMOJI_REGEX',
 'HASTAG_REGEX',
 'MASTERCard_REGEX',
 'MD5_SHA_REGEX',
 'MOST_COMMON_PUNCT_REGEX',
 'NUMBERS_REGEX',
 'PHONE_REGEX',
 'PoBOX_REGEX',
 'SPECIAL_CHARACTERS_REGEX',
 'STOPWORDS',
 'STOPWORDS_de',
 'STOPWORDS_en',
 'STOPWORDS_es',
 'STOPWORDS_fr',
 'STOPWORDS_ru',
 'STOPWORDS_yo',
 'STREET_ADDRESS_REGEX',
 'TextFrame',
 'URL_PATTERN',
 'USER_HANDLES_REGEX',
 'VISACard_REGEX',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__generate_text',
 '__loader__',
 '__name__',
 '__numbers_dict',
 '__package__',
 '__spec__',
 '_lex_richness_herdan',
 '_lex_richness_maas_ttr',
 'clean_text',
 'defaultdict',
 'digit2words',
 'extract_btc_address',
 'extract_currencies',
 'extract_currency_symbols',
 'extract_dates',
 'extract_emails',
 'extract_emojis',
 'extract_hashtags',
 'extract_html_tags',
 'extract_mastercard_addr',
 'extract_md5sha',
 'extract_numbers',
 'extr

In [13]:
kfold = KFold(n_splits=5, random_state=18, shuffle=True)
# Configuration
EPOCHS = 15
BATCH_SIZE = 6
MAX_LEN = 280

# **DATA LOADING**

In [14]:
train_data = pd.read_csv('/kaggle/input/smm4h-2024-task5-classification-of-tweets/SMM4H-2024-Task5-Training.tsv', sep="\t")
train_data

,tweet_id,text,label
0,1364778927105933315,"Psych Med Twitter, wondering if you can help. ...",0
1,1130719434526511104,@dmx_biographer @HellaChillAF I wouldn’t fuck ...,0
2,1184221813733253120,Harley is autistic.... she has the mental capa...,1
3,1435958387020341248,I just made my son cry before going to school ...,1
4,1112559025927999488,Yooooooo my dad really just said “I’d rather h...,0
...,...,...,...
7393,1232778727186223108,Just got all kinds of dirty looks for ordering...,1
7394,1174782065532780544,"anyways- if they did cause autism, i’d rather ...",0
7395,1052612940438728704,2/4 trouble at work and I could have lost my j...,0
7396,1243556662964862976,@LawyerChamber $LgiLder22 I have severe asthma...,1


In [15]:
print("The Shape of the Train Data : ",train_data.shape)

The Shape of the Train Data :  (7398, 3)


In [16]:
train_data['label'].value_counts()

label
0    5118
1    2280
Name: count, dtype: int64

In [17]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7398 entries, 0 to 7397
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  7398 non-null   int64 
 1   text      7398 non-null   object
 2   label     7398 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 173.5+ KB


In [18]:
valid_data = pd.read_csv('/kaggle/input/smm4h-2024-task5-classification-of-tweets/SMM4H-2024-Task5-Validation.tsv', sep='\t')
valid_data

,tweet_id,text,label
0,1265323726570225669,Mom Finds Secret to Homeschooling her Autistic...,0
1,1254180961622900738,I hope our child has my calmness and NOT Andre...,0
2,800336385399885824,Blind Girl With Autism Mimics Whitney Houston ...,0
3,1107948318150152192,"@amyschumer watching #growing, my 2yo son has ...",1
4,1438320313511473154,@robbystarbuck @AmericanAir would rather a mot...,0
...,...,...,...
384,1479160193808273413,my 7yo is being tested for adhd and struggles ...,0
385,1084086161667252225,@attachedmrsL the first reply on the tweet tho...,0
386,1404612356819374080,There's a mom who recorded her autistic son &a...,0
387,1458807608052174856,@itsadollthxng Lmfao girl !!! I’m upset fr cau...,1


In [19]:
print("The Shape of the Validation Data is :",valid_data.shape)

The Shape of the Validation Data is : (389, 3)


In [20]:
valid_data['label'].value_counts()

label
0    254
1    135
Name: count, dtype: int64

In [21]:
valid_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 389 entries, 0 to 388
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  389 non-null    int64 
 1   text      389 non-null    object
 2   label     389 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 9.2+ KB


# **DATA PREPROCESSING**

We see the following anomolies in the tweets
1) There are emojis used in the tweets. </br>
2) The label count is not equal. </br>
3) Use of Hastags. </br>
4) Use of attherate symbol (Tag other users). </br>
5) Use of contractions (like yo-year old, can't - cannot, won't-wont etc...). but roberta is generally robust to contractions so no need of it.</br>

In [22]:
train_data['tweet'] = train_data['text'].apply(nfx.remove_userhandles)
valid_data['tweet'] = valid_data['text'].apply(nfx.remove_userhandles)
print(train_data[:1])
print(valid_data[:1])

              tweet_id                                               text  \
0  1364778927105933315  Psych Med Twitter, wondering if you can help. ...   

   label                                              tweet  
0      0  Psych Med Twitter, wondering if you can help. ...  
              tweet_id                                               text  \
0  1265323726570225669  Mom Finds Secret to Homeschooling her Autistic...   

   label                                              tweet  
0      0  Mom Finds Secret to Homeschooling her Autistic...  


In [23]:
train_data['tweet'] = train_data['tweet'].apply(nfx.remove_hashtags)
valid_data['tweet'] = valid_data['tweet'].apply(nfx.remove_hashtags)
print(train_data[:1])
print(valid_data[:1])

              tweet_id                                               text  \
0  1364778927105933315  Psych Med Twitter, wondering if you can help. ...   

   label                                              tweet  
0      0  Psych Med Twitter, wondering if you can help. ...  
              tweet_id                                               text  \
0  1265323726570225669  Mom Finds Secret to Homeschooling her Autistic...   

   label                                              tweet  
0      0  Mom Finds Secret to Homeschooling her Autistic...  


In [24]:
train_data['tweet'] = train_data['tweet'].apply(nfx.remove_emojis)
valid_data['tweet'] = valid_data['tweet'].apply(nfx.remove_emojis)
print(train_data[:1])
print(valid_data[:1])

              tweet_id                                               text  \
0  1364778927105933315  Psych Med Twitter, wondering if you can help. ...   

   label                                              tweet  
0      0  Psych Med Twitter, wondering if you can help. ...  
              tweet_id                                               text  \
0  1265323726570225669  Mom Finds Secret to Homeschooling her Autistic...   

   label                                              tweet  
0      0  Mom Finds Secret to Homeschooling her Autistic...  


In [25]:
train_data['tweet'] = train_data['tweet'].apply(nfx.remove_urls)
valid_data['tweet'] = valid_data['tweet'].apply(nfx.remove_urls)
print(train_data[:1])
print(valid_data[:1])

              tweet_id                                               text  \
0  1364778927105933315  Psych Med Twitter, wondering if you can help. ...   

   label                                              tweet  
0      0  Psych Med Twitter, wondering if you can help. ...  
              tweet_id                                               text  \
0  1265323726570225669  Mom Finds Secret to Homeschooling her Autistic...   

   label                                              tweet  
0      0  Mom Finds Secret to Homeschooling her Autistic...  


In [26]:
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Remove emojis using the pattern
    return emoji_pattern.sub(r'', text)

In [27]:
train_data['tweet'] = train_data['tweet'].apply(remove_emoji, '')
valid_data['tweet'] = valid_data['tweet'].apply(remove_emoji, '')
print(train_data[:1])
print(valid_data[:1])

              tweet_id                                               text  \
0  1364778927105933315  Psych Med Twitter, wondering if you can help. ...   

   label                                              tweet  
0      0  Psych Med Twitter, wondering if you can help. ...  
              tweet_id                                               text  \
0  1265323726570225669  Mom Finds Secret to Homeschooling her Autistic...   

   label                                              tweet  
0      0  Mom Finds Secret to Homeschooling her Autistic...  


/tmp/ipykernel_34/42957875.py:1: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  train_data['tweet'] = train_data['tweet'].apply(remove_emoji, '')
/tmp/ipykernel_34/42957875.py:2: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  valid_data['tweet'] = valid_data['tweet'].apply(remove_emoji, '')


In [28]:
url_pattern = r'https:\/\/t\.co\/\w+'

train_data['tweet'] = train_data['tweet'].apply(lambda x: re.sub(url_pattern, '', x))
valid_data['tweet'] = valid_data['tweet'].apply(lambda x: re.sub(url_pattern, '', x))

In [29]:
train_data = train_data.drop(['text'], axis=1)
valid_data = valid_data.drop(['text'], axis=1)
print(train_data[:1])
print(valid_data[:1])

              tweet_id  label  \
0  1364778927105933315      0   

                                               tweet  
0  Psych Med Twitter, wondering if you can help. ...  
              tweet_id  label  \
0  1265323726570225669      0   

                                               tweet  
0  Mom Finds Secret to Homeschooling her Autistic...  


In [30]:
train_data.reset_index(drop=True, inplace=True)
valid_data.reset_index(drop=True, inplace=True)

In [31]:
train_data.head(5)

,tweet_id,label,tweet
0,1364778927105933315,0,"Psych Med Twitter, wondering if you can help. ..."
1,1130719434526511104,0,I wouldn’t fuck rob with YOUR dick. He’s a...
2,1184221813733253120,1,Harley is autistic.... she has the mental capa...
3,1435958387020341248,1,I just made my son cry before going to school ...
4,1112559025927999488,0,Yooooooo my dad really just said “I’d rather h...


In [32]:
valid_data.head(5)

,tweet_id,label,tweet
0,1265323726570225669,0,Mom Finds Secret to Homeschooling her Autistic...
1,1254180961622900738,0,I hope our child has my calmness and NOT Andre...
2,800336385399885824,0,Blind Girl With Autism Mimics Whitney Houston ...
3,1107948318150152192,1,"watching my 2yo son has ASD, you just made..."
4,1438320313511473154,0,would rather a mother suffocate an asthmat...


In [33]:
new_train_data = pd.concat([train_data, valid_data], axis=0)
new_train_data.head(5)

,tweet_id,label,tweet
0,1364778927105933315,0,"Psych Med Twitter, wondering if you can help. ..."
1,1130719434526511104,0,I wouldn’t fuck rob with YOUR dick. He’s a...
2,1184221813733253120,1,Harley is autistic.... she has the mental capa...
3,1435958387020341248,1,I just made my son cry before going to school ...
4,1112559025927999488,0,Yooooooo my dad really just said “I’d rather h...


In [34]:
new_train_data.tail(5)

,tweet_id,label,tweet
384,1479160193808273413,0,my 7yo is being tested for adhd and struggles ...
385,1084086161667252225,0,"the first reply on the tweet though yes, I ..."
386,1404612356819374080,0,There's a mom who recorded her autistic son &a...
387,1458807608052174856,1,Lmfao girl !!! I’m upset fr cause jacelyn ha...
388,1299153969009184768,1,I have a child that faces more adver...


In [35]:
new_train_data['label'].value_counts()

label
0    5372
1    2415
Name: count, dtype: int64

In [36]:
print("The shape of the new train data is : ",new_train_data.shape)

The shape of the new train data is :  (7787, 3)


In [37]:
new_train_data.reset_index(drop=True, inplace=True)

In [38]:
new_train_data.head(5)

,tweet_id,label,tweet
0,1364778927105933315,0,"Psych Med Twitter, wondering if you can help. ..."
1,1130719434526511104,0,I wouldn’t fuck rob with YOUR dick. He’s a...
2,1184221813733253120,1,Harley is autistic.... she has the mental capa...
3,1435958387020341248,1,I just made my son cry before going to school ...
4,1112559025927999488,0,Yooooooo my dad really just said “I’d rather h...


In [39]:
new_train_data.tail(5)

,tweet_id,label,tweet
7782,1479160193808273413,0,my 7yo is being tested for adhd and struggles ...
7783,1084086161667252225,0,"the first reply on the tweet though yes, I ..."
7784,1404612356819374080,0,There's a mom who recorded her autistic son &a...
7785,1458807608052174856,1,Lmfao girl !!! I’m upset fr cause jacelyn ha...
7786,1299153969009184768,1,I have a child that faces more adver...


In [40]:
undersampler = RandomUnderSampler(random_state=42)

X_resampled, Y_resampled = undersampler.fit_resample(new_train_data[['tweet_id','tweet']], new_train_data['label'])
new_train_data = pd.DataFrame(X_resampled, columns=['tweet_id','tweet'])
new_train_data['label'] = Y_resampled

print(new_train_data['label'].value_counts())

label
0    2415
1    2415
Name: count, dtype: int64


In [42]:
new_train_data.reset_index(drop=True, inplace=True)

In [43]:
def get_train_dataset(x_data, y_data, batch_size, shuffle=True):
    dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataloader

def get_valid_dataset(x_data, y_data, batch_size):
    dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))
    dataloader = DataLoader(dataset, batch_size=batch_size)
    return dataloader

# **TRAIN THE MODEL**

In [44]:
models = ['roberta-large']

In [45]:
preds = []
weights = []      #f1_score_arrays

In [46]:
class CustomModel(nn.Module):
    def __init__(self, model_name, max_len):
        super(CustomModel, self).__init__()
        self.transformer = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.transformer(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits

In [47]:
def calculate_f1_score(y_true, y_pred):
    true_positives = sum((y_true == 1) & (y_pred == 1))
    false_positives = sum((y_true == 0) & (y_pred == 1))
    false_negatives = sum((y_true == 1) & (y_pred == 0))
    true_negatives = sum((y_true == 0) & (y_pred == 0))
    print("TP's",true_positives)
    print("FP's",false_positives)
    print("FN's",false_negatives)
    print("TN's",true_negatives)
    precision = true_positives / (true_positives + false_positives + 1e-12)
    recall = true_positives / (true_positives + false_negatives + 1e-12)
    
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-12)
    
    return f1_score

In [48]:
def calculate_precision(y_true, y_pred):
    true_positives = sum((y_true == 1) & (y_pred == 1))
    false_positives = sum((y_true == 0) & (y_pred == 1))
    false_negatives = sum((y_true == 1) & (y_pred == 0))
    true_negatives = sum((y_true == 0) & (y_pred == 0))
    precision = true_positives / (true_positives + false_positives + 1e-12)
    
    return precision

In [49]:
def calculate_recall(y_true, y_pred):
    true_positives = sum((y_true == 1) & (y_pred == 1))
    false_positives = sum((y_true == 0) & (y_pred == 1))
    false_negatives = sum((y_true == 1) & (y_pred == 0))
    true_negatives = sum((y_true == 0) & (y_pred == 0))
    recall = true_positives / (true_positives + false_negatives + 1e-12)
    
    return recall

In [50]:
def train_model(model, train_loader, valid_loader, optimizer, criterion, scheduler, fold):
    print("Initializing the loop")
    best_f1_score = -np.inf
    for epoch in range(EPOCHS):
        model.train()
        train_losses = [] 
        # Training loop
        for inputs, targets, attention_mask in train_loader:
            optimizer.zero_grad()
            inputs = inputs.to(device)
            targets = targets.to(device)
            attention_mask = attention_mask.to(device)

            outputs = model(inputs, attention_mask=attention_mask)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()  # Update model parameters

            train_losses.append(loss.item())

        # Validation loop
        model.eval()
        with torch.no_grad():
            valid_losses = []
            valid_preds = []
            valid_targets = []
            for inputs, targets, attention_mask in valid_loader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                attention_mask = attention_mask.to(device)

                outputs = model(inputs, attention_mask=attention_mask)
                loss = criterion(outputs, targets)
                valid_losses.append(loss.item())
                valid_preds.extend(torch.argmax(outputs, axis=1).cpu().numpy())
                valid_targets.extend(targets.cpu().numpy())

            valid_preds = np.array(valid_preds)
            print("The no of valid_preds ",len(valid_preds))
            #print(valid_preds)
            valid_targets = np.array(valid_targets)
            print("The no of valid_targets ",len(valid_targets))
            #print(valid_targets)
            valid_f1_score = calculate_f1_score(valid_targets, valid_preds)
            valid_precision_score = calculate_precision(valid_targets, valid_preds)
            valid_recall_score = calculate_recall(valid_targets, valid_preds)

            avg_train_loss = np.mean(train_losses)
            avg_valid_loss = np.mean(valid_losses)

            print(f"Epoch {epoch + 1}/{EPOCHS}  Train Loss: {avg_train_loss:.4f}  Valid Loss: {avg_valid_loss:.4f}  Valid F1 Score: {valid_f1_score} Precision Score: {valid_precision_score}  Recall Score: {valid_recall_score}")

            # Update best model if necessary
            if valid_f1_score > best_f1_score:
                best_f1_score = valid_f1_score
                torch.save(model.state_dict(), f'fold-{fold}.pt')  # Save model state dictionary

    return best_f1_score

In [51]:
for fold, (train_idx, valid_idx) in enumerate(kfold.split(new_train_data.tweet, new_train_data.label)):
    print('#### FOLD', fold+1)
    x_train, x_valid = new_train_data.tweet[train_idx], new_train_data.tweet[valid_idx]
    y_train, y_valid = new_train_data.label[train_idx], new_train_data.label[valid_idx]

    tokenizer = AutoTokenizer.from_pretrained(models[0])

    # Tokenize data
    x_fake_train = tokenizer(x_train.tolist(), padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
    x_fake_valid = tokenizer(x_valid.tolist(), padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")

    # Convert labels to tensors
    y_fake_train = torch.tensor(y_train.tolist())
    y_fake_valid = torch.tensor(y_valid.tolist())

    # Create datasets and data loaders
    train_dataset = get_train_dataset(x_fake_train, y_fake_train, batch_size=BATCH_SIZE)
    valid_dataset = get_valid_dataset(x_fake_valid, y_fake_valid, batch_size=BATCH_SIZE)

    model = CustomModel(model_name=models[0], max_len=MAX_LEN)
    model = model.to(device)

    # Find optimal learning rate
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=15e-8)  # Set initial learning rate to a very small value

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2, verbose=True)
    best_f1_score = train_model(model, train_dataset, valid_dataset, optimizer, criterion, scheduler, fold)

#### FOLD 1


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Initializing the loop
The no of valid_preds  966
The no of valid_targets  966
TP's 394
FP's 375
FN's 94
TN's 103
Epoch 1/15  Train Loss: 0.7746  Valid Loss: 0.7408  Valid F1 Score: 0.6268894192517117 Precision Score: 0.5123537061118328  Recall Score: 0.8073770491803262
The no of valid_preds  966
The no of valid_targets  966
TP's 318
FP's 285
FN's 170
TN's 193
Epoch 2/15  Train Loss: 0.7430  Valid Loss: 0.7271  Valid F1 Score: 0.582951420714445 Precision Score: 0.5273631840796011  Recall Score: 0.6516393442622938
The no of valid_preds  966
The no of valid_targets  966
TP's 290
FP's 256
FN's 198
TN's 222
Epoch 3/15  Train Loss: 0.7264  Valid Loss: 0.7130  Valid F1 Score: 0.5609284332683592 Precision Score: 0.5311355311355301  Recall Score: 0.5942622950819659
The no of valid_preds  966
The no of valid_targets  966
TP's 402
FP's 321
FN's 86
TN's 157
Epoch 4/15  Train Loss: 0.7177  Valid Loss: 0.6959  Valid F1 Score: 0.6639141205610372 Precision Score: 0.5560165975103727  Recall Score: 0.82

/tmp/ipykernel_34/3253241000.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))
/tmp/ipykernel_34/3253241000.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a

Initializing the loop
The no of valid_preds  966
The no of valid_targets  966
TP's 275
FP's 247
FN's 201
TN's 243
Epoch 1/15  Train Loss: 0.6962  Valid Loss: 0.6901  Valid F1 Score: 0.5511022044083175 Precision Score: 0.5268199233716465  Recall Score: 0.5777310924369735
The no of valid_preds  966
The no of valid_targets  966
TP's 326
FP's 300
FN's 150
TN's 190
Epoch 2/15  Train Loss: 0.6926  Valid Loss: 0.6861  Valid F1 Score: 0.5916515426492358 Precision Score: 0.5207667731629384  Recall Score: 0.6848739495798305
The no of valid_preds  966
The no of valid_targets  966
TP's 288
FP's 197
FN's 188
TN's 293
Epoch 3/15  Train Loss: 0.6808  Valid Loss: 0.6668  Valid F1 Score: 0.5993756503637027 Precision Score: 0.5938144329896895  Recall Score: 0.6050420168067214
The no of valid_preds  966
The no of valid_targets  966
TP's 318
FP's 171
FN's 158
TN's 319
Epoch 4/15  Train Loss: 0.6608  Valid Loss: 0.6318  Valid F1 Score: 0.659067357512452 Precision Score: 0.6503067484662564  Recall Score: 0.

/tmp/ipykernel_34/3253241000.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))
/tmp/ipykernel_34/3253241000.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a

Initializing the loop
The no of valid_preds  966
The no of valid_targets  966
TP's 330
FP's 292
FN's 163
TN's 181
Epoch 1/15  Train Loss: 0.6981  Valid Loss: 0.6936  Valid F1 Score: 0.5919282511205818 Precision Score: 0.5305466237942114  Recall Score: 0.6693711967545625
The no of valid_preds  966
The no of valid_targets  966
TP's 437
FP's 381
FN's 56
TN's 92
Epoch 2/15  Train Loss: 0.6967  Valid Loss: 0.6896  Valid F1 Score: 0.6666666666661963 Precision Score: 0.5342298288508551  Recall Score: 0.8864097363083145
The no of valid_preds  966
The no of valid_targets  966
TP's 468
FP's 399
FN's 25
TN's 74
Epoch 3/15  Train Loss: 0.6896  Valid Loss: 0.6834  Valid F1 Score: 0.6882352941171839 Precision Score: 0.539792387543252  Recall Score: 0.949290060851925
The no of valid_preds  966
The no of valid_targets  966
TP's 372
FP's 208
FN's 121
TN's 265
Epoch 4/15  Train Loss: 0.6752  Valid Loss: 0.6540  Valid F1 Score: 0.6933830382101264 Precision Score: 0.6413793103448264  Recall Score: 0.75456

# **PREDICTION**

In [52]:
new_valid_data = pd.read_csv("/kaggle/input/smm4h-2024-task5-classification-of-tweets/SMM4H-2024-Task5-Validation.tsv", sep="\t")
new_valid_data

,tweet_id,text,label
0,1265323726570225669,Mom Finds Secret to Homeschooling her Autistic...,0
1,1254180961622900738,I hope our child has my calmness and NOT Andre...,0
2,800336385399885824,Blind Girl With Autism Mimics Whitney Houston ...,0
3,1107948318150152192,"@amyschumer watching #growing, my 2yo son has ...",1
4,1438320313511473154,@robbystarbuck @AmericanAir would rather a mot...,0
...,...,...,...
384,1479160193808273413,my 7yo is being tested for adhd and struggles ...,0
385,1084086161667252225,@attachedmrsL the first reply on the tweet tho...,0
386,1404612356819374080,There's a mom who recorded her autistic son &a...,0
387,1458807608052174856,@itsadollthxng Lmfao girl !!! I’m upset fr cau...,1


In [53]:
new_valid_data['tweet'] = new_valid_data['text'].apply(nfx.remove_userhandles)
new_valid_data['tweet'] = new_valid_data['tweet'].apply(nfx.remove_hashtags)
new_valid_data['tweet'] = new_valid_data['tweet'].apply(nfx.remove_emojis)
new_valid_data['tweet'] = new_valid_data['tweet'].apply(nfx.remove_urls)
new_valid_data['tweet'] = new_valid_data['tweet'].apply(remove_emoji,'')
new_valid_data = new_valid_data.drop(['text'], axis=1)
new_valid_data.reset_index(drop=True, inplace=True)

/tmp/ipykernel_34/1883816501.py:5: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  new_valid_data['tweet'] = new_valid_data['tweet'].apply(remove_emoji,'')


In [54]:
new_valid_data

,tweet_id,label,tweet
0,1265323726570225669,0,Mom Finds Secret to Homeschooling her Autistic...
1,1254180961622900738,0,I hope our child has my calmness and NOT Andre...
2,800336385399885824,0,Blind Girl With Autism Mimics Whitney Houston ...
3,1107948318150152192,1,"watching my 2yo son has ASD, you just made..."
4,1438320313511473154,0,would rather a mother suffocate an asthmat...
...,...,...,...
384,1479160193808273413,0,my 7yo is being tested for adhd and struggles ...
385,1084086161667252225,0,"the first reply on the tweet though yes, I ..."
386,1404612356819374080,0,There's a mom who recorded her autistic son &a...
387,1458807608052174856,1,Lmfao girl !!! I’m upset fr cause jacelyn ha...


In [55]:
tweets = new_valid_data.tweet
labels = new_valid_data.label

In [56]:
tokenized_tweets = tokenizer(tweets.tolist(), padding=True, truncation=True, max_length=MAX_LEN, return_tensors="pt")
encoded_label = torch.tensor(labels.tolist())

In [57]:
valid_pred_dataset = get_valid_dataset(tokenized_tweets, encoded_label, batch_size=BATCH_SIZE)

/tmp/ipykernel_34/3253241000.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset = TensorDataset(torch.tensor(x_data['input_ids']), torch.tensor(y_data), torch.tensor(x_data['attention_mask']))


In [58]:
model_1 = CustomModel(model_name=models[0], max_len=MAX_LEN)
model_1.load_state_dict(torch.load("/kaggle/working/fold-1.pt"))
model_1 = model_1.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [59]:
def predict(model, dataloader):
    predictions = []
    model.eval()
    with torch.no_grad():
        for input_ids,_,attention_mask in dataloader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            predictions.extend(torch.argmax(outputs, dim=1).cpu().numpy())
    return predictions

In [64]:
predictions = predict(model_1,valid_pred_dataset)

In [85]:
# predictions

In [65]:
f1_score = calculate_f1_score(np.array(encoded_label.tolist()), np.array(predictions))
print("f1_score is : ",f1_score)
precision = calculate_precision(np.array(encoded_label.tolist()), np.array(predictions))
print("precision is : ", precision)
recall = calculate_recall(np.array(encoded_label.tolist()), np.array(predictions))
print(" recall is : ", recall)

TP's 128
FP's 26
FN's 7
TN's 228
f1_score is :  0.8858131487884233
precision is :  0.8311688311688258
 recall is :  0.9481481481481412


In [62]:
df = pd.DataFrame()

In [63]:
df = pd.DataFrame({'tweet_id': new_valid_data['tweet_id'], 'label': predictions})
df.to_csv("prediction_task5.tsv", sep="\t", index=False)

In [141]:
del model_1